# 1. 与 google drive 链接，导入文件

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

dir_path = '/content/drive/MyDrive/2023NLPCourse/Assignment1/Part_B'
file_name = 'Data/data.csv'

%cd /content/drive/MyDrive/2023NLPCourse/Assignment1/Part_B

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/2023NLPCourse/Assignment1/Part_B


# 2. 使用pandas读取评论数据文件 data.csv

In [2]:
import pandas as pd
df = pd.read_csv(file_name)
df = df.dropna()

# 3. 数据清洗，此处只是移除了文本中存在的表情

In [3]:
!pip install clean-text
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize
from cleantext import clean

new_df = pd.DataFrame()

review = []
review_id = []

for index in df.index:
    sentences = sent_tokenize(df.loc[index, 'review'])
    for sentence in sentences:
        review.append(clean(sentence, no_emoji=True))
        review_id.append(index)

new_df['review_id'] = review_id
new_df['review'] = review

# 4. 定义文本预处理函数

In [4]:
import string
import emoji
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

stopwords = nltk.corpus.stopwords.words('english')

def preprocess(text):
    
    # 1.text blob 移除一些符号（过程，抽取词语，然后拼接词语）
    text_blob = TextBlob(text)
    text = ' '.join(text_blob.words)
    
    # 2. 移除文本中的数字
    text = re.sub(r'[0-9]', '', text)
    
    # 3. 最小化文本
    text = text.lower()
    
    # 4. 移除标点符号
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    
    # 5. 分词
    text = word_tokenize(text)
    
    # 6. 移除分词后存在的空白token 
    text = [t for t in text if len(t) > 0]
    
    # 7. 移除非英文字符
    text = [t for t in text if t.isalpha()]
    
    # 8. 替换否定表达形式的 token，减少模型理解难度
    replacer  = AntonymReplacer()
    text = replacer.replace_negations(text)
    
    # 9. 移除停用词
    text = [i for i in text if i not in stopwords]
    
    # 10. 词性还原
    text = lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    return text


def lemmatization(sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(sent)) 
    texts_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return texts_out

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word == 'not' and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 5. 对输入数据进行预处理

In [ ]:
cleaned_text = []
counter = 1

for text in new_df["review"]:
    print(f'Preprocessing row {counter}')
    counter+=1
    cleaned_text.append(preprocess(str(text)))

new_df["Clean"] = cleaned_text
new_df = new_df.dropna()

new_df.to_excel("Data/preprocessed_data.xlsx")
print('Preprocessing done.')

# 6. LDA模型训练

In [ ]:
# 以下生成的 extracted_data.xlsx 是为 Bertopic 准备的数据，Bertopic模型数据输入上限为 100K，故只保存了100k
    # 经过预处理后，获得的数据数量为 120k左右，只减少了 20k 尽量保证数据本身不受影响
pd.read_excel("Data/preprocessed_data.xlsx").sample(n=100000).to_excel("Data/extracted_data.xlsx", index=False)
extracted_data = pd.read_excel("Data/preprocessed_data.xlsx")


import gensim.corpora as corpora
import gensim.models as models

from ast import literal_eval
from pprint import pprint

data_words = []
for x in extracted_data['Clean']:
    data_words.append(literal_eval(x))

# 创建词典
id2word = corpora.Dictionary(data_words)
# 创建语料库
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]

# LDA模型选择的主题数量，以及每个主题的word数量
num_topics = 36
num_words = 10

# 根据词典，语料库，以及主题数量训练 LDA 主题模型
lda_model1 = models.LdaMulticore(corpus=corpus,
                              id2word=id2word,
                              num_topics=num_topics)

# 7. 保存模型

In [ ]:
import pickle
topics = lda_model1.print_topics(num_topics=num_topics, num_words=num_words)
topic_list = []
for topic in topics:
    topic_list.append(topic[1])

df_topics = pd.DataFrame(topic_list,  columns =['topics'])

df_topics.to_excel("Data/LDA_topics.xlsx")
pickle.dump(lda_model1, open('Models/LDA_model.model', 'wb'))   # save lda model
pickle.dump(id2word, open('Models/LDA_model.dict', 'wb'))       # save dictionary
pickle.dump(corpus, open('Models/LDA_model.corpus', 'wb'))      # save corpus

# 8.LDA主题模型结果的可视化结果展示

In [ ]:
import pickle

loaded_lda_model = pickle.load(open('Models/LDA_model.model', 'rb'))  # load trained model
loaded_dictionary = pickle.load(open('Models/LDA_model.dict', 'rb'))  # load dictionary 
loaded_corpus = pickle.load(open('Models/LDA_model.corpus', 'rb'))    # load corpus

!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

In [6]:
vis = gensimvis.prepare(loaded_lda_model, loaded_corpus, loaded_dictionary)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
9     -0.059390  0.029400       1        1  5.033487
20     0.084147 -0.000384       2        1  4.659285
11    -0.191670  0.001404       3        1  3.994857
26     0.025550 -0.042588       4        1  3.827145
7      0.018893  0.107232       5        1  3.295044
3      0.090226  0.104762       6        1  3.290838
17     0.070628  0.041608       7        1  3.271331
29    -0.226943  0.080839       8        1  3.083164
34     0.063211  0.112621       9        1  3.053589
16    -0.237310  0.132601      10        1  3.010589
30    -0.304755  0.076998      11        1  2.992921
22     0.030613  0.060413      12        1  2.931236
5     -0.229291  0.050167      13        1  2.907268
10     0.096099  0.086354      14        1  2.877985
13    -0.109657 -0.180123      15        1  2.837567
27     0.078608  0.077019      16        1  2.756557
25     0.088068  0.101347      17        1  2.739051
18     0.130207  0.083743      18        1  2.694945
31     0.046251  0.040780      19        1  2.584270
19    -0.034768 -0.068347      20        1  2.581078
0      0.118308  0.027024      21        1  2.536237
6     -0.016819 -0.151498      22        1  2.521146
12     0.039336  0.061864      23        1  2.482594
33     0.028888 -0.040559      24        1  2.422706
1      0.094280 -0.020150      25        1  2.402779
14     0.076547  0.036628      26        1  2.400394
2      0.098104  0.005835      27        1  2.381341
28    -0.096973  0.004119      28        1  2.364222
32    -0.021310 -0.232672      29        1  2.341704
24     0.079649  0.032424      30        1  2.331196
35     0.010662 -0.035585      31        1  2.175699
8      0.072918  0.012583      32        1  2.095249
21     0.010098 -0.209152      33        1  2.007895
15    -0.015089 -0.154229      34        1  1.913795
4      0.041489 -0.191568      35        1  1.780848
23     0.051197 -0.040910      36        1  1.419987, topic_info=         Term          Freq         Total Category  logprob  loglift
36      great  11062.000000  11062.000000  Default  30.0000  30.0000
3        good  11294.000000  11294.000000  Default  29.0000  29.0000
102       buy   9975.000000   9975.000000  Default  28.0000  28.0000
34        fit  18809.000000  18809.000000  Default  27.0000  27.0000
21       size   8879.000000   8879.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1335  reorder    209.077520    223.786047  Topic36  -3.6476   4.1865
585     value    523.525364   2550.695287  Topic36  -2.7297   2.6710
78    quality    352.484327   6826.685350  Topic36  -3.1253   1.2909
2       shirt    279.848823  28384.031516  Topic36  -3.3561  -0.3648
122       get     48.905097   5615.627863  Topic36  -5.1004  -0.4889

[1390 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
158       8  0.994001        able
880       2  0.996297  absolutely
872      15  0.984692      accord
519      27  0.974023     account
692      10  0.992915    accurate
...     ...       ...         ...
2007     31  0.984545   yesterday
264       2  0.410526         yet
264      13  0.584430         yet
1756     11  0.939163       young
2060     36  0.985064          yr

[1866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 21, 12, 27, 8, 4, 18, 30, 35, 17, 31, 23, 6, 11, 14, 28, 26, 19, 32, 20, 1, 7, 13, 34, 2, 15, 3, 29, 33, 25, 36, 9, 22, 16, 5, 24])